# PPA

In [ ]:
import os,sys; sys.path.insert(0,os.path.abspath('../..'))
from contxt import *
from ppa import *
from zipfile import ZipFile
ht_data_dir = '/Users/ryan/data/ppa/ht_text_pd'
pagestr="""
112 
 
 it down as a kind of rule,* that " the several 
 " inflexions which form the ess^^ncc of speaking 
 ** and reading, may be considered as two only? 
 " the rising and falling inflexions." 
 
 
 M; 
 
 '•h 
 
 But surely, to confound things so diiferent as 
 simple and compotuid accents, is to mislead instead 
 of instructing the learner. We have in modern^ 
 as doubtless they had in ancient languages, under 
 the general name of circumflex accer>ts, all the 
 varieties that can arise from tlie combination of 
 two opposite inflexions, in the utterance of 
 single svllables. Our circumflexes are not only 
 distinguishable as h^ixiggra'DC-acitte and acuto-grave^ 
 according to the order in which their elementary 
 slides are combined, but also as consisting of 
 elements which vary in their respective dimen- 
 sions. To indicate these, in Mr. Steele's simple 
 method above mentioned, by drawing the accen- 
 tual marks over the syllables, three marks at least 
 will be requisite for each class, viz. ^v^ v_-^ v^ 
 grave-acuteywndi-^ ^~^ ^ acutc-grave. Nor have we 
 yet specified all the varieties which are perceptible 
 in our tones of speech. We have two classes of 
 combined inflexions in a double circumflex, which 
 
 * P. M6. 
"""
pd.DataFrame(iter_sents_in_str(pagestr))

: 

In [2]:

class PPACorpus(BaseCorpus):
    id='ppa'
    name='PPA'
    
    def raw_sents(self, progress=True, desc='Compiling corpus'):
        files = list(sorted(os.walk(ht_data_dir)))
        iterr=get_tqdm(iterable=files, desc=desc) if progress else files
        for root,dirs,fns in iterr:
            fn_zip = first(fn for fn in fns if fn.endswith('.zip'))
            if fn_zip:
                fnfn_zip =os.path.join(ht_data_dir,root,fn_zip)
                with ZipFile(fnfn_zip) as zip:
                    namelist=list(sorted(list(zip.namelist())))
                    # Loop over each file
                    for member in namelist:
                        if '/' in member and member.endswith('.txt'):
                            id,pageid=member[:-4].split('/')
                            if pageid.isdigit():
                                page_i=int(pageid)
                                with zip.open(member) as f:
                                    txt=f.read().decode(encoding='utf-8')
                                    for sentd in iter_sents_in_str(txt):
                                        yield dict(
                                            _id=f'{id}/page/{page_i}/sent/{page_i}',
                                            text_id=id,
                                            corpus_id=self.id,
                                            page_i=page_i,
                                            **sentd
                                        )
    
    def compile_sents(self, gen=None, batchsize=1000):
        if gen is None: gen=self.raw_sents()
        def _addbatch(batch): 
            with DataDB() as dbx:
                dbx.sents_text.bulk_write(batch)
        workers = get_workers()
        batch=[]
        for sentd in gen:
            batch.append(
                ReplaceOne(
                    dict(_id=sentd['_id']),
                    sentd,
                    upsert=True
                )
            )
            if len(batch)>=batchsize: 
                workers.apply_async(_addbatch, (batch,))
                batch=[]
        if batch: workers.apply_async(_addbatch, (batch,))
        return

In [3]:
C=PPACorpus()
C.id

'ppa'

In [4]:
C.compile_sents()

creating threadpool with 20 workers


Compiling corpus: 100%|██████████| 16583/16583 [56:54<00:00,  4.86it/s]  


In [6]:
df=pd.DataFrame(islice(DB().sents_text.find(dict(corpus_id='ppa')), 1000))


In [7]:
df

,_id,text_id,corpus_id,page_i,para_i,sent_i,sent
0,ark+=13960=t1hh7pc2q/page/0/sent/0,ark+=13960=t1hh7pc2q,ppa,0,5,5,123 WEST MAIN STREET
1,ark+=13960=t1hh7pc2q/page/1/sent/1,ark+=13960=t1hh7pc2q,ppa,1,3,3,Canadian Institute for Historical Microreprodu...
2,ark+=13960=t1hh7pc2q/page/2/sent/2,ark+=13960=t1hh7pc2q,ppa,2,39,44,y
3,ark+=13960=t1hh7pc2q/page/21/sent/21,ark+=13960=t1hh7pc2q,ppa,21,3,8,B 5
4,ark+=13960=t1hh7pc2q/page/7/sent/7,ark+=13960=t1hh7pc2q,ppa,7,15,16,"BUDD,CROWN AND MITRE^PALL MALL} By G. Sidney, ..."
...,...,...,...,...,...,...,...
995,086569286/page/20/sent/20,086569286,ppa,20,0,2,"Table of Initial Articulative Combinations, Ta..."
996,086569286/page/21/sent/21,086569286,ppa,21,0,2,"PAGE an, 66 Section First-VOWELS, First Vowel,..."
997,086569286/page/22/sent/22,086569286,ppa,22,0,3,"ARTICULATIVE EXERCISES, LITERAL EXERCISES- Bre..."
998,086569286/page/23/sent/23,086569286,ppa,23,0,1,"PAGE VOCAL PHYSIOLOGY -- VISIBLE SPEECH, . VIS..."
